In [1]:
import pandas as pd
import numpy as np
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_csv('data/data-mentah.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df[df['author_location'].isnull() == False]
df = df[df['author_location'].isna() == False]

df = df.head(10)
df.head(5)

,tweet_id,author_name,author_username,author_location,content,content_clean,like_count,retweet_count,reply_count,source,...,latitude,province,city,country,created_at,content_token,tweet_tokens_fdist,tweet_tokens_WSW,tweet_tokens_stemmed,tweet_tokens_stemmed_string
0,153619597713608704,MariBerlomba,MariBerlomba,di dalam asa pemuda,"1. lomba poster ilmiah ""energi baru/terbarukan...",lomba poster ilmiah energi baru terbarukan dea...,0.0,1.0,NaN,Twitter for BlackBerry®,...,NaN,NaN,NaN,NaN,2012-01-01 23:32:20,"['lomba', 'poster', 'ilmiah', 'energi', 'baru'...",<FreqDist with 13 samples and 13 outcomes>,"['lomba', 'poster', 'ilmiah', 'energi', 'terba...","['lomba', 'poster', 'ilmiah', 'energi', 'baru'...",lomba poster ilmiah energi baru deadline janua...
1,153857491925610496,vegaaulia,vegaaulia,"Bogor, Indonesia",elaahhh ini si bapak tiba2 ngomong soal energi...,elaahhh ini si bapak tiba ngomong soal energi ...,0.0,NaN,1.0,Mobile Web,...,-6.596299,Jawa Barat,Kelurahan Pabaton,Indonesia,2012-01-02 15:17:39,"['elaahhh', 'ini', 'si', 'bapak', 'tiba', 'ngo...",<FreqDist with 13 samples and 13 outcomes>,"['elaahhh', 'ngomong', 'energi', 'terbarukan',...","['elaahhh', 'ngomong', 'energi', 'baru', 'tv',...",elaahhh ngomong energi baru tv biruu
2,154361424154603520,enbater,enbater,Indonesia,masih banyak daerah terpencil membutuhkam list...,masih banyak daerah terpencil membutuhkam list...,0.0,NaN,NaN,MOTOBLUR,...,-2.483383,Kalimantan Timur,NaN,Indonesia,2012-01-04 00:40:06,"['masih', 'banyak', 'daerah', 'terpencil', 'me...",<FreqDist with 13 samples and 13 outcomes>,"['daerah', 'terpencil', 'membutuhkam', 'listri...","['daerah', 'pencil', 'membutuhkam', 'listrik',...",daerah pencil membutuhkam listrik manfaat ener...
3,156397219346518017,imronrosidin,imronrosidin,"ÜT: -6.895087,107.627799",gw kira kalo @provocactive gak tertarik dgn ma...,gw kira kalo gak tertarik dgn materi energi ba...,0.0,NaN,1.0,UberSocial for BlackBerry,...,NaN,NaN,NaN,NaN,2012-01-09 15:29:37,"['gw', 'kira', 'kalo', 'gak', 'tertarik', 'dgn...",<FreqDist with 18 samples and 18 outcomes>,"['tertarik', 'materi', 'energi', 'terbarukan',...","['tarik', 'materi', 'energi', 'baru', 'baek', ...",tarik materi energi baru baek ganti nama propa...
4,156936339718279168,revozone,revozone,Indonesia,satu lagi yang aneh. inget jatropa? alias miny...,satu lagi yang aneh inget jatropa alias minyak...,0.0,NaN,NaN,UberSocial for BlackBerry,...,-2.483383,Kalimantan Timur,NaN,Indonesia,2012-01-11 03:11:53,"['satu', 'lagi', 'yang', 'aneh', 'inget', 'jat...",<FreqDist with 20 samples and 21 outcomes>,"['aneh', 'inget', 'jatropa', 'alias', 'minyak'...","['aneh', 'inget', 'jatropa', 'alias', 'minyak'...",aneh inget jatropa alias minyak jarak bbrp jat...


In [3]:
def lookup_location(location):
    geo_locator = Nominatim(user_agent="LearnPython")
    try:
        location = geo_locator.geocode(location, language='en')
    except GeocoderTimedOut:
        return None
    except:
        return None
    return location

In [4]:
def lookup_location_by_coordinate(latitude, longitude):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        location = geolocator.reverse([latitude+","+longitude])
    except GeocoderTimedOut:
        return None
    except:
        return None
    return location

In [5]:
for index, row in df.iterrows():
    if row['author_location']:
        location = lookup_location(row['author_location'])
        if location is not None:
            df.loc[index, 'address'] = location.address
            df.loc[index, 'latitude'] = location.latitude
            df.loc[index, 'longitude'] = location.longitude

            # coordinate
            location_detail = lookup_location_by_coordinate(location.latitude,location.longitude )
            if location_detail:
                address = location_detail.raw['address']
                country = address.get('country', '')
                province = address.get('state', '')
                city = address.get('city', '')
                
                if country:
                    print(country, province, city)
                    df.loc[index, 'country'] = country
                    df.loc[index, 'province'] = province
                    df.loc[index, 'city'] = city
        else:
            df.loc[index, 'address'] = ""

In [6]:
df.to_csv('data/data-geocoding.csv')